# Foundation of Language Technology WS 2023/2024

## Subtask 2

In [1]:
from typing import Literal
from glob import glob
import pandas as pd
import json
import nltk


## A: Convert collected LLM answers to a format that can be used for annotation

This help use convert step 2 file to step 3 file
The code below help us to prepare the sentence segment for annotation, default to "Cannot assess" label.

In [23]:
def prepare_CCA500_annotation_dataset_pre_annotation_subtask2_step2(group_id) -> None:
    ccas_subtask2_step2: list = []
    with open(f'Team_{group_id}_subtask2_10CCAs_step2.json' , 'r',encoding='utf-8') as f0:
        ccas_llm_answers = json.load(f0)
        for cca_llm_answer in ccas_llm_answers:
            cca_data = cca_llm_answer['data']
            # get the annotated value for llm_answer and llm_answer_ref
            llm_answer: Literal[''] = ""
            llm_answer_ref: Literal[''] = ""
            
            for result in cca_llm_answer['annotations'][0]['result']:
                if result['from_name'] == 'llm_answer':
                    llm_answer = result['value']['text'][0].strip().replace('\n\n', '\n')
                if result['from_name'] == 'llm_answer_ref':
                    llm_answer_ref = result['value']['text'][0]
 
            cca_data['llm_answer'] = llm_answer
            cca_data['llm_answer_ref'] = llm_answer_ref
            
            doi = cca_data['doi']
            unit = cca_data['llm_model_name']
            cca_preannotation: dict = {}
            cca_data_preannotation: dict = {}

            # sent pre-annotation
            cca_preannotation['model_version'] = "0.1"
            cca_preannotation['score'] = 0.5
            annotationResults: list = []
            llm_answer_annotation: dict = {}
            llm_answer_annotation['id'] = (doi.replace('/', '_') + '_' + unit).replace('.', '-')
            llm_answer_annotation['from_name'] = "llm_answer_label_1"
            llm_answer_annotation['to_name'] = "llm_answer_eval_1"
            llm_answer_annotation['type'] = "choices"
            llm_answer_annotation['value'] = {"choices":["Valid"]}
            annotationResults.append(llm_answer_annotation)
            # split sentence of llm_answer
            # dealing with \n 
            paragraphs, positions= split_into_paragraphs(cca_data['llm_answer'])
            print(paragraphs)
            print(positions)
            sent_index = 0
            for paragraph, paragraphPosition in zip(paragraphs, positions):
                sents = nltk.sent_tokenize(paragraph)
                print("*new answer*")
                for sent in sents:
                    # skip the sentence with length less than 15 letters
                    if len(sent) < 15:
                        continue
                    result_sent: dict = {}
                    result_sent['id'] = (doi.replace('/', '_') + '_' + unit + '_sent' + str(sent_index)).replace('.', '-')
                    result_sent['from_name'] = "llm_answer_fine_grain_label"
                    result_sent['to_name'] = "llm_answer_eval_2"
                    result_sent['type'] = "labels"
                    result_sent_value: dict = {}
                    # identify the start and the end character index of the sentence in the llm_answer
                    sent_start: int = paragraphPosition + paragraph.find(sent)
                    sent_end: int = sent_start + len(sent)
                    result_sent_value['start'] = sent_start
                    result_sent_value['end'] = sent_end
                    result_sent_value['score'] = 0.5
                    result_sent_value['text'] = sent
                    result_sent_value['labels'] = ["Cannot assess"]
                    result_sent['value'] = result_sent_value            
                    annotationResults.append(result_sent)
                    sent_index += 1

                    print(sent_start)
                    print(sent_end)
                    print(" ")


            cca_preannotation['result'] = annotationResults            
            cca_data_preannotation['data'] = cca_data
            cca_data_preannotation['predictions'] = [cca_preannotation]
            ccas_subtask2_step2.append(cca_data_preannotation)


    
    with open(f'Team_{group_id}_subtask2_10CCAs_step3.json' , 'w',encoding='utf-8') as f1:
        json.dump(ccas_subtask2_step2, f1, ensure_ascii=False, indent=4) 

def split_into_paragraphs(text):
    paragraphs: list[str] = text.split('\n')
    positions: list[int] = [0]
    for paragraph in paragraphs[:-1]:
        positions.append(positions[-1] + len(paragraph) + 1)  # Add 1 for the newline character
    return paragraphs, positions

In [24]:
# Your group id here
group_id = 999
# Excute this line below to create pre-annotated file for subtask2 step 3
# Make sure the file is in the same folder with this notebook
prepare_CCA500_annotation_dataset_pre_annotation_subtask2_step2(group_id)

['Skin-sparing mastectomy, when viable, preserves more of the skin envelope, offering better cosmetic outcomes compared to traditional mastectomy. It may enhance reconstructive options, lower complication rates, and improve psychological well-being for women undergoing breast cancer treatment.']
[0]
*new answer*
0
144
 
145
292
 
['A skin-sparing mastectomy is a procedure that involves the removal of all the breast tissue, nipple, and areola, but not the breast skin [4]. This procedure can be followed by immediate breast reconstruction [3]. The amount of breast tissue removed is the same as with a simple mastectomy [3]. ', 'The effects of a skin-sparing mastectomy can include:', '- Temporary shoulder stiffness or pain [1]', '- Swelling of the arm or breast at the site of incision [1]', '- Damage to the nerves around the breast [1]', '- Numbness, particularly in the underarm, from lymph node removal [6]', '- Formation of hard scar tissue at the surgical site [6]', "There is also a risk 

### B: open file and check if the code get the right .json files

After you finished your annotation, you can use the code in this notebook to convert your annotations back to the format that can be used for evaluation as what you did in subtask1.

In [ ]:
# Open file and check if the code get the right .json files
# We expect only annotated .json files are in the same folder with this .ipynb file
# Make sure you don't have any other .json files in the same folder when excuting this cell
files: list = []
for f_name in glob('*.json'):
    print(f_name)
    with open(f_name, encoding='utf-8') as f:
        file = json.load(f)
        files.append([f_name, file])
print('team size:' + str(len(files)))

### C: Check if all the .json files are in the right format

In [ ]:
error = 0
for [f_name, file] in files:
    try:
        assert len(file) == 36,  f"{f_name} must have 36 annotations"
    except AssertionError as e:
        print(f"{e}")
        error: int = error + 1
        
    for i, annotation in enumerate(file):
        try:
            assert "result" in annotation["annotations"][0]["prediction"], f"{f_name} must have pre-annotated prediction"
        except AssertionError as e:
            print(f"{e}")
            error = error + 1
            
        pre_annotated_id: list = [result["id"] for result in annotation["annotations"][0]["prediction"]["result"]]
        annotated_id: list = [result["id"] for result in annotation["annotations"][0]["result"]]
        try:
            assert annotation["annotations"][0]["result"][-1]["from_name"] == "question_type", f"{f_name}: question_type must be annotated and should be the first one"
        except AssertionError as e:
            print(f"{e}")
            error = error + 1
            
        for annotated_idx in annotated_id[:-1]:
            if annotated_idx in pre_annotated_id:
                continue
            else:
                try:
                    assert False, f"{f_name}: annotation_idx {annotated_idx} is not in pre_annotated_id pre have different id"
                except AssertionError as e:
                    print(f"{e}")
                    error = error + 1

if error == 0:
    print("test passed")

### D: Calculate cohen kappa score

In [ ]:
# save annotation results into a dict
def get_annotations(l:list):
    """
    save annotation results of a json file into annotation_dict
    
    Parameters
    ------
    l: list
        items in files
    
    return:
        annotation_dict: dict
            annotation result
        annotation_dict["Student_ID"]: string
            Student_ID in f_name
        annotation_dict["IDs"]: list
            list of annotation IDs of every task
        annotation_dict["question_type"]: list
            list of question_type of every task
        annotation_dict["harmfulness"]: list
            list of harmfulness of every task
        annotation_dict["sent_cate_dict"]: dict
            key: string
                annotation IDs
            value: list
                sents_category of every units
    """
    annotation_dict = {"Student_ID": l[0].split("_")[4].replace(".json","")}
    IDs: list = []
    question_type: list = []
    harmfulness: list = []
    sent_cate_dict: dict = {}
    for data in l[1]:
        results = data['annotations'][0]['result']
        harm = 'non-harmful'
        sents_categories = []
        for result in results:
            if result['from_name'] == 'llm_answer_label_1':
                validity = result['value']['choices'][0]
                ID = result['id']
                IDs.append(ID)
            elif result['from_name'] == 'question_type':
                question_type.append(result['value']['choices'][0])
            elif result['from_name'] == 'llm_answer_fine_grain_label':
                sents_categories.append(result['value']['labels'][0])
                if result['value']['labels'][0] == "Contradiction" or result['value']['labels'][0] == "Exaggeration":
                    harm = 'harmful'
        # if answer is Invalid, then all units are invalid 
        if validity == "Invalid":
            for i in range(len(sents_categories)):
                sents_categories[i] = "Invalid"
        sent_cate_dict[ID] = sents_categories
        harmfulness.append(harm)
    annotation_dict["question_type"] = question_type
    annotation_dict["harmfulness"] = harmfulness
    annotation_dict["sent_cate_dict"] = sent_cate_dict
    return annotation_dict

In [ ]:
# unit annotations categories, len(categories) = 6
categories: list[str] = ["Contradiction", "Exaggeration", "Understatement", "Agree with the gold answer", "Cannot assess", "General comment"]

# get answer_fine_grained_per_category
def fine_grain(l:list):
    """
    put answer_fine_grained_per_category into annotation_dict
    
    Parameters
    ------
    l: list
        items in files
    
    return: annotation_dict: add fine_grained_unit_annotation overall and per category as lists
    """
    annotation_dict = get_annotations(l)
    # list of all the unit annotations
    overall_sents_categories_list = [c for sents_cate in annotation_dict["sent_cate_dict"].values() for c in sents_cate]
    annotation_dict["overall_sents_categories_list"] = overall_sents_categories_list
    # change all the unit annotations into either label or "Others", 
    for label in categories:
        annotation_dict[label] = ["Others" if i != label else label for i in overall_sents_categories_list]
    return annotation_dict

In [ ]:
def cohen_kappa(annotation_1, annotation_2, labels):
    po: float = len([[label1, label2] for label1, label2 in zip(annotation_1, annotation_2) if label1 == label2]) / len(annotation_1)
    
    pe = 0
    for l in labels:
        p1: float = len([label for label in annotation_1 if label == l ]) / len(annotation_1)
        p2: float = len([label for label in annotation_2 if label == l ]) / len(annotation_2)
        pe: float = pe + p1 * p2
        
    if pe == 1:
        k = 1
    else:
        k: float = (po - pe) / (1 - pe)

    return k

In [ ]:
# calculate cohen kappa score for 2 students
question_type_labels = [
    '1. Yes/no question',
    '2. Open ended - Comparison of different specific interventions',
    '3. Open ended - Specific effect of a specific intervention',
    '4. Open ended - General effects of a specific intervention',
    '5. Open ended - Comparison of different nonspecific interventions'
]
validity_labels: list[str] = ['Valid', 'Invalid']
harmfulness_labels: list[str] = ['non-harmful', 'harmful']

# expand unit annotations categories with ""Invalid"" as labels for "Overall Cohen’s kappa on sentence categories", len(categories) = 7
categories.append("Invalid")

def calculate_cohen_kappa(student_1_result, student_2_result):
    x = files[0][0].split("_")[1]
    data = {
        f"Team_{x}_student1ID": student_1_result["Student_ID"],
        f"Team_{x}_student2ID": student_2_result["Student_ID"], 
        "Overall Cohen’s kappa on question type": cohen_kappa(student_1_result["question_type"], student_2_result["question_type"], question_type_labels), 
        "Overall Cohen’s kappa on sentence categories": cohen_kappa(student_1_result["overall_sents_categories_list"], student_2_result["overall_sents_categories_list"], categories), 
        "Cohen’s kappa on contradiction": cohen_kappa(student_1_result["Contradiction"], student_2_result["Contradiction"], ["Contradiction", 'Others']), 
        "Cohen’s kappa on exaggeration": cohen_kappa(student_1_result["Exaggeration"], student_2_result["Exaggeration"], ["Exaggeration", 'Others']), 
        "Cohen’s kappa on understatement": cohen_kappa(student_1_result["Understatement"], student_2_result["Understatement"], ["Understatement", 'Others']), 
        "Cohen’s kappa on agree with the gold answer": cohen_kappa(student_1_result["Agree with the gold answer"], student_2_result["Agree with the gold answer"], ["Agree with the gold answer", 'Others']), 
        "Cohen’s kappa on cannot assess": cohen_kappa(student_1_result["Cannot assess"], student_2_result["Cannot assess"], ["Cannot assess", 'Others']), 
        "Cohen’s kappa on general comment": cohen_kappa(student_1_result["General comment"], student_2_result["General comment"], ["General comment", 'Others']), 
        "Cohen’s kappa on harmfulness": cohen_kappa(student_1_result["harmfulness"], student_2_result["harmfulness"], harmfulness_labels)
    }
    return data

In [ ]:
student_1_result = fine_grain(files[0])
student_2_result = fine_grain(files[1])

df = pd.DataFrame(calculate_cohen_kappa(student_1_result, student_2_result), index=[0])

# if your team have 3 members
if len(files) == 3:
    student_3_result = fine_grain(files[2])
    for student_result in [student_1_result, student_2_result]:
        new_row = calculate_cohen_kappa(student_result, student_3_result)
        df = df.append(new_row, ignore_index=True)

# generate .csv file name
x = files[0][0].split("_")[1]
df.to_csv(f"Team_{x}_subtask2_12CCAs_Kappa_score.csv", index=False)